# MULTICLASS CLASSIFICAION(Kaggle_California_crime)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#MULTICLASS-CLASSIFICAION(Kaggle_California_crime)" data-toc-modified-id="MULTICLASS-CLASSIFICAION(Kaggle_California_crime)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>MULTICLASS CLASSIFICAION(Kaggle_California_crime)</a></span><ul class="toc-item"><li><span><a href="#LOGISTIC-REGRESSION-(classification)" data-toc-modified-id="LOGISTIC-REGRESSION-(classification)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>LOGISTIC REGRESSION (classification)</a></span><ul class="toc-item"><li><span><a href="#EVALUATION" data-toc-modified-id="EVALUATION-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>EVALUATION</a></span></li><li><span><a href="#PREDICTION-CONVERTED-BACK-TO-LABEL" data-toc-modified-id="PREDICTION-CONVERTED-BACK-TO-LABEL-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>PREDICTION CONVERTED BACK TO LABEL</a></span></li><li><span><a href="#CROSS-VALIDATION" data-toc-modified-id="CROSS-VALIDATION-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>CROSS VALIDATION</a></span></li><li><span><a href="#PRINT-PARAMS" data-toc-modified-id="PRINT-PARAMS-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>PRINT PARAMS</a></span></li></ul></li><li><span><a href="#NAIVE-BAYES-(classification)" data-toc-modified-id="NAIVE-BAYES-(classification)-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>NAIVE BAYES (classification)</a></span></li><li><span><a href="#RANDOM-FOREST-(classification)" data-toc-modified-id="RANDOM-FOREST-(classification)-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>RANDOM FOREST (classification)</a></span><ul class="toc-item"><li><span><a href="#PRINT-PARAMS" data-toc-modified-id="PRINT-PARAMS-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>PRINT PARAMS</a></span></li></ul></li><li><span><a href="#NOTE:-LinearSVC-supports-binary-classification-only???" data-toc-modified-id="NOTE:-LinearSVC-supports-binary-classification-only???-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>NOTE: LinearSVC supports binary classification only???</a></span></li><li><span><a href="#One-vs-Rest-(classification)" data-toc-modified-id="One-vs-Rest-(classification)-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>One-vs-Rest (classification)</a></span></li><li><span><a href="#GRADIENT-BOOSTED-TREE" data-toc-modified-id="GRADIENT-BOOSTED-TREE-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>GRADIENT-BOOSTED TREE</a></span></li><li><span><a href="#MULTILAYER-PERCEPTRON" data-toc-modified-id="MULTILAYER-PERCEPTRON-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>MULTILAYER PERCEPTRON</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#EXAMPLE" data-toc-modified-id="EXAMPLE-1.7.0.1"><span class="toc-item-num">1.7.0.1&nbsp;&nbsp;</span>EXAMPLE</a></span></li></ul></li><li><span><a href="#trainingData-split" data-toc-modified-id="trainingData-split-1.7.1"><span class="toc-item-num">1.7.1&nbsp;&nbsp;</span>trainingData split</a></span></li></ul></li></ul></li></ul></div>

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MULTICLASS CLASSIFICAION").getOrCreate()
spark

In [2]:
train_data_file= '/Users/pinky/Downloads/DATA/Kaggle/california_crime/train.csv'
test_data_file= '/Users/pinky/Downloads/DATA/Kaggle/california_crime/test.csv'

In [3]:
trainDF = spark.read.format('csv').option('header','true').load(train_data_file)

In [4]:
trainDF.show(5)

+-------------------+--------------+--------------------+---------+----------+--------------+--------------------+-------------------+------------------+
|              Dates|      Category|            Descript|DayOfWeek|PdDistrict|    Resolution|             Address|                  X|                 Y|
+-------------------+--------------+--------------------+---------+----------+--------------+--------------------+-------------------+------------------+
|2015-05-13 23:53:00|      WARRANTS|      WARRANT ARREST|Wednesday|  NORTHERN|ARREST, BOOKED|  OAK ST / LAGUNA ST|  -122.425891675136|  37.7745985956747|
|2015-05-13 23:53:00|OTHER OFFENSES|TRAFFIC VIOLATION...|Wednesday|  NORTHERN|ARREST, BOOKED|  OAK ST / LAGUNA ST|  -122.425891675136|  37.7745985956747|
|2015-05-13 23:33:00|OTHER OFFENSES|TRAFFIC VIOLATION...|Wednesday|  NORTHERN|ARREST, BOOKED|VANNESS AV / GREE...|   -122.42436302145|  37.8004143219856|
|2015-05-13 23:30:00| LARCENY/THEFT|GRAND THEFT FROM ...|Wednesday|  NORTHER

In [5]:
trainDF2 = trainDF.select('Category','Descript')

In [6]:
trainDF2.show(3)

+--------------+--------------------+
|      Category|            Descript|
+--------------+--------------------+
|      WARRANTS|      WARRANT ARREST|
|OTHER OFFENSES|TRAFFIC VIOLATION...|
|OTHER OFFENSES|TRAFFIC VIOLATION...|
+--------------+--------------------+
only showing top 3 rows



In [7]:
trainDF2.printSchema()

root
 |-- Category: string (nullable = true)
 |-- Descript: string (nullable = true)



In [8]:
#distinct categories 
trainDF2.groupby('Category').count().sort('count', ascending=False).show(5, truncate=False)

+--------------+------+
|Category      |count |
+--------------+------+
|LARCENY/THEFT |174900|
|OTHER OFFENSES|126182|
|NON-CRIMINAL  |92304 |
|ASSAULT       |76876 |
|DRUG/NARCOTIC |53971 |
+--------------+------+
only showing top 5 rows



In [9]:
#Analyze the data

In [10]:
trainDF2.groupby('Descript').count().sort('count', ascending=False).show(5,truncate=False)

+-------------------------------------+-----+
|Descript                             |count|
+-------------------------------------+-----+
|GRAND THEFT FROM LOCKED AUTO         |60022|
|LOST PROPERTY                        |31729|
|BATTERY                              |27441|
|STOLEN AUTOMOBILE                    |26897|
|DRIVERS LICENSE, SUSPENDED OR REVOKED|26839|
+-------------------------------------+-----+
only showing top 5 rows



## LOGISTIC REGRESSION (classification)

In [11]:
# Pipeline: Tokenizer --> StopWordsRemover --> CountVectorizer
# StringIndexer for label column

In [12]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, StringIndexer
from pyspark.ml.pipeline import Pipeline

In [13]:
default_stop_words = StopWordsRemover().getStopWords()

In [14]:
type(default_stop_words)

list

In [15]:
new_stop_words = default_stop_words.copy()
new_stop_words.append('')
type(new_stop_words)
new_stop_words.append('-')

In [16]:
new_stop_words[len(new_stop_words)-10:]

["that's",
 "there's",
 "what's",
 "when's",
 "where's",
 "who's",
 "why's",
 'would',
 '',
 '-']

In [17]:
tokenizer = Tokenizer(inputCol='Descript', outputCol='words')
stopwordsremover = StopWordsRemover(inputCol='words', outputCol='words_cleansed', stopWords=new_stop_words)
vectorizer = CountVectorizer(inputCol='words_cleansed', outputCol='features')

label_stringIndexer = StringIndexer(inputCol='Category', outputCol='label')

In [18]:
pipeline = Pipeline(stages=[tokenizer, stopwordsremover, vectorizer, label_stringIndexer])

In [19]:
tranformedDF = pipeline.fit(trainDF2).transform(trainDF2)

In [20]:
type(tranformedDF)

pyspark.sql.dataframe.DataFrame

In [21]:
tranformedDF.show(10)

+--------------+--------------------+--------------------+--------------------+--------------------+-----+
|      Category|            Descript|               words|      words_cleansed|            features|label|
+--------------+--------------------+--------------------+--------------------+--------------------+-----+
|      WARRANTS|      WARRANT ARREST|   [warrant, arrest]|   [warrant, arrest]|(1055,[11,23],[1....|  7.0|
|OTHER OFFENSES|TRAFFIC VIOLATION...|[traffic, violati...|[traffic, violati...|(1055,[8,11,26],[...|  1.0|
|OTHER OFFENSES|TRAFFIC VIOLATION...|[traffic, violati...|[traffic, violati...|(1055,[8,11,26],[...|  1.0|
| LARCENY/THEFT|GRAND THEFT FROM ...|[grand, theft, fr...|[grand, theft, lo...|(1055,[0,1,2,3],[...|  0.0|
| LARCENY/THEFT|GRAND THEFT FROM ...|[grand, theft, fr...|[grand, theft, lo...|(1055,[0,1,2,3],[...|  0.0|
| LARCENY/THEFT|GRAND THEFT FROM ...|[grand, theft, fr...|[grand, theft, un...|(1055,[0,1,2,92],...|  0.0|
| VEHICLE THEFT|   STOLEN AUTOMOBILE|

In [22]:
tranformedDF.select('Category').distinct().collect()

[Row(Category='FRAUD'),
 Row(Category='SUICIDE'),
 Row(Category='SEX OFFENSES FORCIBLE'),
 Row(Category='LIQUOR LAWS'),
 Row(Category='SECONDARY CODES'),
 Row(Category='FAMILY OFFENSES'),
 Row(Category='MISSING PERSON'),
 Row(Category='OTHER OFFENSES'),
 Row(Category='DRIVING UNDER THE INFLUENCE'),
 Row(Category='WARRANTS'),
 Row(Category='ARSON'),
 Row(Category='SEX OFFENSES NON FORCIBLE'),
 Row(Category='FORGERY/COUNTERFEITING'),
 Row(Category='GAMBLING'),
 Row(Category='BRIBERY'),
 Row(Category='ASSAULT'),
 Row(Category='DRUNKENNESS'),
 Row(Category='EXTORTION'),
 Row(Category='TREA'),
 Row(Category='WEAPON LAWS'),
 Row(Category='LOITERING'),
 Row(Category='SUSPICIOUS OCC'),
 Row(Category='ROBBERY'),
 Row(Category='PROSTITUTION'),
 Row(Category='EMBEZZLEMENT'),
 Row(Category='BAD CHECKS'),
 Row(Category='DISORDERLY CONDUCT'),
 Row(Category='RUNAWAY'),
 Row(Category='RECOVERED VEHICLE'),
 Row(Category='VANDALISM'),
 Row(Category='DRUG/NARCOTIC'),
 Row(Category='PORNOGRAPHY/OBSCENE MA

In [23]:
tranformedDF.select('words_cleansed').distinct().sort('words_cleansed').show(5,truncate=False)

+-----------------------------------------+
|words_cleansed                           |
+-----------------------------------------+
|[abandonment, child]                     |
|[abortion]                               |
|[access, card, information,, publication]|
|[access, card, information,, theft]      |
|[accidental, burns]                      |
+-----------------------------------------+
only showing top 5 rows



In [24]:
tranformedDF.select('words_cleansed').distinct().sort('words_cleansed', ascending=False).show(5,truncate=False)

+--------------------------------------------+
|words_cleansed                              |
+--------------------------------------------+
|[youth, court]                              |
|[willful, cruelty, child]                   |
|[wearing, mask, disguise, unlawful, purpose]|
|[wearing, apparel, opposite, sex, deceive]  |
|[weapons, possession, juvenile, suspect]    |
+--------------------------------------------+
only showing top 5 rows



In [25]:
# vectorizer.params
# tokenizer.params
# stopwordsremover.params

In [26]:
print(stopwordsremover.getStopWords()[:50])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do']


In [27]:
trainingData, testData = tranformedDF.randomSplit([0.7, 0.3])

In [28]:
trainingData.count(), testData.count()

(614478, 263571)

In [29]:
from pyspark.ml.classification import LogisticRegression
logisticRegression = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = logisticRegression.fit(trainingData)

In [30]:
#print(lrModel.explainParams())

In [31]:
predictions = lrModel.transform(testData)

In [32]:
predictions.columns

['Category',
 'Descript',
 'words',
 'words_cleansed',
 'features',
 'label',
 'rawPrediction',
 'probability',
 'prediction']

In [33]:
predictions.select(['label', 'prediction']).show(5)

+-----+----------+
|label|prediction|
+-----+----------+
| 27.0|       0.0|
| 27.0|       0.0|
| 27.0|       0.0|
| 27.0|       0.0|
| 27.0|       0.0|
+-----+----------+
only showing top 5 rows



In [34]:
predictions.show(10)

+--------+--------+-------+--------------+------------------+-----+--------------------+--------------------+----------+
|Category|Descript|  words|words_cleansed|          features|label|       rawPrediction|         probability|prediction|
+--------+--------+-------+--------------+------------------+-----+--------------------+--------------------+----------+
|   ARSON|   ARSON|[arson]|       [arson]|(1055,[196],[1.0])| 27.0|[3.02305577241817...|[0.12781878150593...|       0.0|
|   ARSON|   ARSON|[arson]|       [arson]|(1055,[196],[1.0])| 27.0|[3.02305577241817...|[0.12781878150593...|       0.0|
|   ARSON|   ARSON|[arson]|       [arson]|(1055,[196],[1.0])| 27.0|[3.02305577241817...|[0.12781878150593...|       0.0|
|   ARSON|   ARSON|[arson]|       [arson]|(1055,[196],[1.0])| 27.0|[3.02305577241817...|[0.12781878150593...|       0.0|
|   ARSON|   ARSON|[arson]|       [arson]|(1055,[196],[1.0])| 27.0|[3.02305577241817...|[0.12781878150593...|       0.0|
|   ARSON|   ARSON|[arson]|     

In [35]:
predictions.groupBy('label').count().sort('count').show(10)

+-----+-----+
|label|count|
+-----+-----+
| 38.0|    2|
| 37.0|    6|
| 36.0|   39|
| 35.0|   48|
| 34.0|   69|
| 33.0|   86|
| 32.0|  126|
| 30.0|  146|
| 31.0|  147|
| 29.0|  330|
+-----+-----+
only showing top 10 rows



In [36]:
predictions.groupBy('Category').count().sort('count').show(10)

+--------------------+-----+
|            Category|count|
+--------------------+-----+
|                TREA|    2|
|PORNOGRAPHY/OBSCE...|    6|
|            GAMBLING|   39|
|SEX OFFENSES NON ...|   48|
|           EXTORTION|   69|
|             BRIBERY|   86|
|          BAD CHECKS|  126|
|             SUICIDE|  146|
|     FAMILY OFFENSES|  147|
|        EMBEZZLEMENT|  330|
+--------------------+-----+
only showing top 10 rows



### EVALUATION

In [133]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
mEvaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
lrModel_accuracy = mEvaluator.evaluate(predictions)
print('lrModel_accuracy:', lrModel_accuracy)
#O: 0.974708090565633   # The accuracy is excellent!
#O2:0.980403762174139
#O3: lrModel_accuracy: 0.980403762174139 

mEvaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='weightedPrecision')
lrModel_weightedPrecision = mEvaluator.evaluate(predictions)
print('lrModel_weightedPrecision:' , lrModel_weightedPrecision)
#O: lrModel_weightedPrecision: 0.97492545179755

mEvaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='weightedRecall')
lrModel_weightedRecall = mEvaluator.evaluate(predictions)
print('lrModel_weightedRecall:' , lrModel_weightedRecall)
lrModel_weightedRecall: 0.9804037621741389
    
mEvaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='f1')
lrModel_f1 = mEvaluator.evaluate(predictions)
print('lrModel_f1:' , lrModel_f1)
lrModel_f1: 0.9748541010853148
 

lrModel_accuracy: 0.980403762174139
lrModel_weightedPrecision: 0.97492545179755
lrModel_weightedPrecision: 0.97492545179755
lrModel_weightedRecall: 0.9804037621741389
lrModel_f1: 0.9748541010853148


### PREDICTION CONVERTED BACK TO LABEL  

In [ ]:
label_and_preductions_only = predictions.select('Category', 'label','prediction')
label_list = 'FRAUD', 'SUICIDE', 'SEX OFFENSES FORCIBLE', 'LIQUOR LAWS', 'SECONDARY CODES', 'FAMILY OFFENSES', 'MISSING PERSON', 'OTHER OFFENSES', 'DRIVING UNDER THE INFLUENCE', 'WARRANTS', 'ARSON', 'SEX OFFENSES NON FORCIBLE', 'FORGERY/COUNTERFEITING', 'GAMBLING', 'BRIBERY', 'ASSAULT', 'DRUNKENNESS', 'EXTORTION', 'TREA', 'WEAPON LAWS', 'LOITERING', 'SUSPICIOUS OCC', 'ROBBERY', 'PROSTITUTION', 'EMBEZZLEMENT', 'BAD CHECKS', 'DISORDERLY CONDUCT', 'RUNAWAY', 'RECOVERED VEHICLE', 'VANDALISM', 'DRUG/NARCOTIC', 'PORNOGRAPHY/OBSCENE MAT', 'TRESPASS', 'VEHICLE THEFT', 'NON-CRIMINAL', 'STOLEN PROPERTY', 'LARCENY/THEFT', 'KIDNAPPING','BURGLARY'


In [ ]:
from pyspark.ml.feature import IndexToString
indexToStringTransformer = IndexToString(inputCol='prediction', outputCol='prediction_Converted', labels=label_list) 
result = indexToStringTransformer.transform(label_and_preductions_only)

In [ ]:
result.show(4)

In [ ]:
result.groupBy('Category','prediction_Converted').count().sort('count').show(truncate=False)

In [ ]:
result.groupBy('Category','prediction_Converted').sum().show(truncate=False)

In [ ]:
result.groupBy('Category').count().show(truncate=False)

In [ ]:
result.groupBy('prediction_Converted').count().show(truncate=False)

In [ ]:
trainingData.count(), testData.count()

In [ ]:
result.count()

In [ ]:
result.show(5)

In [ ]:
result.groupBy('label','prediction').count().sort('prediction').show(truncate=False)

In [ ]:
result.groupBy('Category','prediction_Converted').count().show(truncate=False)

### CROSS VALIDATION

In [38]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [39]:
paramGrid = ParamGridBuilder()

In [40]:
# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(logisticRegression.regParam, [0.1, 0.3, 0.5]) # regularization parameter
             .addGrid(logisticRegression.elasticNetParam, [0.0, 0.1, 0.2]) # Elastic Net Parameter (Ridge = 0)
#            .addGrid(model.maxIter, [10, 20, 50]) #Number of iterations
#            .addGrid(idf.numFeatures, [10, 100, 1000]) # Number of features
             .build())

In [41]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")

In [42]:
cv = CrossValidator(estimator=logisticRegression, evaluator=evaluator, estimatorParamMaps=paramGrid, numFolds=5)

In [43]:
cvModel = cv.fit(trainingData)
cv_predictions = cvModel.transform(testData)

In [67]:
cv_predictions.show(3)

+--------+--------+-------+--------------+------------------+-----+--------------------+--------------------+----------+
|Category|Descript|  words|words_cleansed|          features|label|       rawPrediction|         probability|prediction|
+--------+--------+-------+--------------+------------------+-----+--------------------+--------------------+----------+
|   ARSON|   ARSON|[arson]|       [arson]|(1055,[196],[1.0])| 27.0|[2.77053080296224...|[0.07351631415968...|      27.0|
|   ARSON|   ARSON|[arson]|       [arson]|(1055,[196],[1.0])| 27.0|[2.77053080296224...|[0.07351631415968...|      27.0|
|   ARSON|   ARSON|[arson]|       [arson]|(1055,[196],[1.0])| 27.0|[2.77053080296224...|[0.07351631415968...|      27.0|
+--------+--------+-------+--------------+------------------+-----+--------------------+--------------------+----------+
only showing top 3 rows



In [68]:
cv_evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction',metricName='accuracy')

In [69]:
cv_result = cv_evaluator.evaluate(cv_predictions)

In [71]:
cv_result  
#O:0.9952851156305446 #The performance improved.
#O2: 0.9956178790534619

0.9956178790534619

In [72]:
bestModel = cvModel.bestModel

In [73]:
bestModel

LogisticRegression_45ff9fe1573f1926e73e

In [74]:
bestModel_params = bestModel.extractParamMap()
bestModel_params.items()

dict_items([(Param(parent='LogisticRegression_45ff9fe1573f1926e73e', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2)'), 2), (Param(parent='LogisticRegression_45ff9fe1573f1926e73e', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty'), 0.0), (Param(parent='LogisticRegression_45ff9fe1573f1926e73e', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial.'), 'auto'), (Param(parent='LogisticRegression_45ff9fe1573f1926e73e', name='featuresCol', doc='features column name'), 'features'), (Param(parent='LogisticRegression_45ff9fe1573f1926e73e', name='fitIntercept', doc='whether to fit an intercept term'), True), (Param(parent='LogisticRegression_45ff9fe1573f1926e73e', name='labelCol', doc='label column name'), 'label'), (Param(parent='LogisticRegression_45ff9fe15

In [75]:
bestModel_params.values()

dict_values([2, 0.0, 'auto', 'features', True, 'label', 20, 'prediction', 'probability', 'rawPrediction', 0.1, True, 0.5, 1e-06])

In [76]:
bestModel.getParam('elasticNetParam')

Param(parent='LogisticRegression_45ff9fe1573f1926e73e', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty')

In [77]:
bestModel.getParam('regParam')

Param(parent='LogisticRegression_45ff9fe1573f1926e73e', name='regParam', doc='regularization parameter (>= 0)')

### PRINT PARAMS

In [78]:
for key, value in bestModel.extractParamMap().items():
    print (key.name, ":", value)

aggregationDepth : 2
elasticNetParam : 0.0
family : auto
featuresCol : features
fitIntercept : True
labelCol : label
maxIter : 20
predictionCol : prediction
probabilityCol : probability
rawPredictionCol : rawPrediction
regParam : 0.1
standardization : True
threshold : 0.5
tol : 1e-06


In [55]:
 betsModel_predictions = bestModel.transform(testData)
#cv_evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction',metricName='accuracy')
cv_evaluator.evaluate(betsModel_predictions)
#O: 0.9952851156305446 # SAME RESULT as cvModel_predictions

0.9956178790534619

## NAIVE BAYES (classification)

In [56]:
from pyspark.ml.classification import NaiveBayes

In [57]:
naiveBayes = NaiveBayes()
                             #featuresCol='features', 
                             #labelCol='label',
                             #predictionCol='', 
                             #modelType='',  #"multinomial" (default) and "bernoulli"
                             #probabilityCol='',
                             #rawPredictionCol='',
                             #smoothing='',  #smoothing parameter. Default is 1.0.
                             #self='', 
                             #thresholds='', 
                             #weightCol='', #default: set or not empty; so all instances have weight=1
                             #lambda='')   #Set the smoothing parameter. Default: 1.0.

In [58]:
naiveBayesModel = naiveBayes.fit(trainingData)

In [59]:
naiveBayesPredictions = naiveBayesModel.transform(testData)

In [60]:
#naiveBayesPredictions.orderBy('probability').sort('probability').show(3)
#naiveBayesPredictions.orderBy('probability').sort('probability', ascenging=False).show(3, truncate=False)

In [61]:
naiveBayesPredictions.printSchema()

root
 |-- Category: string (nullable = true)
 |-- Descript: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- words_cleansed: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- label: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [62]:
naiveBayesEvaluator = MulticlassClassificationEvaluator(predictionCol='prediction')

In [63]:
naiveBayesEvaluator.evaluate(naiveBayesPredictions)
# O: 0.993997341990219   # Good, but not better than LogisticRegression Cross validator

0.9940657249526593

## RANDOM FOREST (classification)

In [64]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import RandomForestParams

In [65]:
randomForest = RandomForestClassifier(numTrees=100, maxBins=32, maxDepth=4)
#                                       numTrees='',  #(default = 20) 
                                                      #If 1, then no bootstrapping is used. 
#                                                     #If > 1, then bootstrapping is done
                                                      #cannot use this property in GBT or 
                                                      #RF(TreeEnsembleParams) because maxIter controls this
#                                       impurity='',  #Criterion used for information gain calculation (case-insensitive).
                                                      #gini" (recommended) or "entropy"
#                                       maxBins='',   # (default = 32)
                                                      #Maximum number of bins used for discretizing continuous features and for choosing 
                                                      #how to split on features at each node. More bins give 
                                                      #higher granularity. Must be >= 2 and >= number of categories 
                                                      #in any categorical feature. 
#                                       maxDepth='',  #(suggested value: 4)
#                                       checkpointInterval='', #set checkpoint interval (>= 1) or disable checkpoint (-1). 
                                                               #Ex: 10 means that the cache will get checkpointed every 10 iterations.
#                                       featureSubsetStrategy='', #(default = "auto")
                                                                  #The number of features to consider for splits at each tree node
                                                                  # auto | all | onethird | sqrt | log2 | n * number of features, with n = range(0.0,1.0)
                                                                  # auto: 1 if numTress = 1; 
                                                                  # all :use all features
                                                                  # onethird: use one third of the features
                                                                  # sqrt: sqrt(number of features); recommended by Breiman manual for random forests
                                                                  # log2: log2(number of features)

#                                       maxMemoryInMB='',
#                                       minInfoGain='', #(default = 0.0)
                                                        #Minimum information gain for a split to be considered at a tree node. 
                                                        #Should be >= 0.0. 
#                                       minInstancesPerNode='', #(default = 1)
                                                                #Minimum number of instances each child must have after split. 
                                                                #If a split causes the left or right child to have fewer than minInstancesPerNode, 
                                                                #the split will be discarded as invalid. Should be >= 1
#                                       predictionCol='',
#                                       seed='',
#                                       subsamplingRate='', #Fraction of the training data used for learning 
                                                            #each decision tree, in range (0, 1]. (default = 1.0)
#                                       featuresCol='',
#                                       labelCol='',
#                                       probabilityCol='',
#                                       rawPredictionCol='',
#                                       thresholds='')   # adjust the probability of predicting each class

In [66]:
type(randomForestModel)

NameError: name 'randomForestModel' is not defined

In [ ]:
#display(randomForestModel) # works with DATABRICKS only)

In [ ]:
print(display(randomForestModel))

In [ ]:
randomForestModel = randomForest.fit(trainingData)
randomForestPredictions = randomForestModel.transform(testData)

In [ ]:
randomForestModel.params

In [ ]:
randomForestEvaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
randomForestEvaluator.evaluate(predictions)

Random forest is a very good, robust and versatile method, however it’s no mystery that for high-dimensional sparse data it’s not a best choice.

It is obvious that Logistic Regression will be our model in this experiment, with cross validation.

In [ ]:
val params = cvModel3Day.bestModel.asInstanceOf[PipelineModel].stages(2).asInstanceOf[GBTClassificationModel].extractParamMap()

val depth = cvModel3Day.bestModel.asInstanceOf[PipelineModel].stages(2).asInstanceOf[GBTClassificationModel].getMaxDepth

### PRINT PARAMS

In [ ]:
params = randomForestModel.extractParamMap()

In [ ]:
params.values()

In [ ]:
#[lambda x: param.values(x) for x in params.keys()]

In [ ]:
for key, value in randomForestModel.extractParamMap().items():
    print(key.name, ':', value)

In [ ]:
params2 = randomForestModel.getOrDefault('maxMemoryInMB')

In [ ]:
params2

## NOTE: LinearSVC supports binary classification only???

## One-vs-Rest (classification)

In [ ]:
from pyspark.ml.classification import OneVsRest, LogisticRegression

In [ ]:
lr = LogisticRegression(maxIter=10, tol=1E-6, fitIntercept=True)

In [ ]:
ovr = OneVsRest(classifier=lr)

In [ ]:
ovrModel = ovr.fit(trainingData)
ovr_predictions = ovrModel.transform(testData)

In [ ]:
ovr_predictions

In [ ]:
ovrEvaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
ovrEvaluator.evaluate(predictions)

## GRADIENT-BOOSTED TREE

## MULTILAYER PERCEPTRON

    (MLPC) is a classifier based on the feedforward artificial neural network. MLPC consists of multiple layers of nodes. Each layer is fully connected to the next layer in the network. Nodes in the input layer represent the input data. All other nodes map inputs to outputs by a linear combination of the inputs with the node’s weights w and bias b and applying an activation function
    The number of nodes N in the output layer corresponds to the number of classes.MLPC employs backpropagation for learning the model. We use the logistic loss function for optimization and L-BFGS as an optimization routine.

#### EXAMPLE

In [79]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [82]:
#my_layers = [4, 5, 4, 3]
#mPerceptron = MultilayerPerceptronClassifier(layers=my_layers, blockSize=128, maxIter=100, seed=1234)
trainer = MultilayerPerceptronClassifier(layers=[856, 5, 4, 39])

In [83]:
mPerceptronModel = trainer.fit(trainingData)

In [84]:
mPerceptronPredictions = mPerceptronModel.transform(testData)

In [87]:
mPerceptronPredictions

DataFrame[Category: string, Descript: string, words: array<string>, words_cleansed: array<string>, features: vector, label: double, rawPrediction: vector, probability: vector, prediction: double]

In [ ]:
mPerceptronEvaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
mPerceptronPredictions.evaluate(predictions)

In [ ]:
mr_parameter_grid_ = ParamGridBuilder().addGrid(mr.maxIter, [50, 200, 500])\
    .addGrid(lr.regParam, [0, 0.3, 1])\
    .addGrid(lr.elasticNetParam, [0, 0.3, 1]).build()

In [97]:
from pyspark.ml.linalg import Vectors
df = spark.createDataFrame([
      (0.0, Vectors.dense([0.0, 0.0])),
      (1.0, Vectors.dense([0.0, 1.0])),
      (1.0, Vectors.dense([1.0, 0.0])),
      (0.0, Vectors.dense([1.0, 1.0]))], ["label", "features"])
mlp = MultilayerPerceptronClassifier(maxIter=100, layers=[2, 2, 2], blockSize=1, seed=123)
model = mlp.fit(df)
testDF = spark.createDataFrame([
      (Vectors.dense([1.0, 0.0]),),
      (Vectors.dense([0.0, 0.0]),)], ["features"])
model.transform(testDF).select("features", "prediction").show()

+---------+----------+
| features|prediction|
+---------+----------+
|[1.0,0.0]|       1.0|
|[0.0,0.0]|       0.0|
+---------+----------+



In [98]:
trainingData.printSchema()

root
 |-- Category: string (nullable = true)
 |-- Descript: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- words_cleansed: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- label: double (nullable = false)



In [99]:
trainingData.groupby('Category').count().sort('count', ascending=False).show(truncate=False)

+----------------------+------+
|Category              |count |
+----------------------+------+
|LARCENY/THEFT         |122363|
|OTHER OFFENSES        |88319 |
|NON-CRIMINAL          |64260 |
|ASSAULT               |53837 |
|DRUG/NARCOTIC         |37826 |
|VEHICLE THEFT         |37608 |
|VANDALISM             |31385 |
|WARRANTS              |29733 |
|BURGLARY              |25771 |
|SUSPICIOUS OCC        |22024 |
|MISSING PERSON        |18214 |
|ROBBERY               |16098 |
|FRAUD                 |11631 |
|FORGERY/COUNTERFEITING|7338  |
|SECONDARY CODES       |6999  |
|WEAPON LAWS           |5987  |
|PROSTITUTION          |5253  |
|TRESPASS              |5110  |
|STOLEN PROPERTY       |3139  |
|SEX OFFENSES FORCIBLE |3057  |
+----------------------+------+
only showing top 20 rows



### trainingData split

In [108]:
# ERROR: trainingData.filter(trainingData['Category'] in ('PROSTITUTION','TRESPASS','TRESPASS'))
trainingData2,_=trainingData.randomSplit([0.1, 0.9])
trainingData2.count(),trainingData.count()

(61479, 614478)

In [111]:
testData2,_=testData.randomSplit([0.1, 0.9])
testData2.count(),testData.count()

(26457, 263571)

In [126]:
trainer = MultilayerPerceptronClassifier(layers=[856, 5, 4, 662])
mPerceptronModel = trainer.fit(trainingData2)

In [112]:
mPerceptronPredictions = mPerceptronModel.transform(testData2)
mPerceptronPredictions

DataFrame[Category: string, Descript: string, words: array<string>, words_cleansed: array<string>, features: vector, label: double, rawPrediction: vector, probability: vector, prediction: double]

In [114]:
trainingData2.columns

['Category', 'Descript', 'words', 'words_cleansed', 'features', 'label']

In [129]:
 #trainingData2.select('Category').distinct().collect() #Row count:39
 trainingData2.select('Descript').distinct().count()

662